In [ ]:
import pandas as pd
import os

#https://www.analyticsvidhya.com/blog/2021/04/animated-bar-graph-data-science-project/

In [ ]:
# reference https://ec.europa.eu/eurostat/databrowser/view/EI_ISBU_Q__custom_6200052/settings_1/table?lang=en
df = pd.read_csv('Data/ei_isbu_q__custom_6200052_linear.csv', header=0,  parse_dates=True)

In [ ]:
df.head()



In [ ]:
#remove rows where 'OBS_VALUE' is null
df = df[df['OBS_VALUE'].notna()]

In [ ]:
#drop column 'LAST UPDATE'
df = df.drop(columns=['LAST UPDATE'])
#drop column 'FREQ'
df = df.drop(columns=['freq'])
#drop column 'DATAFLOW'
df = df.drop(columns=['DATAFLOW'])
#drop column 's_adj'
df = df.drop(columns=['s_adj'])
#drop column 'unit'
df = df.drop(columns=['unit'])
#drop column 'nace_r2'
df = df.drop(columns=['nace_r2'])
#drop column 'OBS_FLAG'
df = df.drop(columns=['OBS_FLAG'])

df.head(5)



In [ ]:
#create a column for each unique value in 'indic'setting its row value =  to 'OBS_VALUE'
df = pd.get_dummies(df, columns=['indic'])
#remove 'indic_' in column names
df.columns = df.columns.str.replace('indic_', '')
#if the value in 'IS-EPI' or 'IS-HWI' or 'IS-IP' or 'IS-WSI' = 1, set its value to 'OBS_VALUE'
df['IS-EPI'] = df['IS-EPI'] * df['OBS_VALUE']
df['IS-HWI'] = df['IS-HWI'] * df['OBS_VALUE']
df['IS-IP'] = df['IS-IP'] * df['OBS_VALUE']
df['IS-WSI'] = df['IS-WSI'] * df['OBS_VALUE']





df.set_index('TIME_PERIOD', inplace=True)

In [ ]:
df = df.drop(columns=['OBS_VALUE'])
df.head(5)

In [ ]:
#remove all rows where 'IS-EPI' = 0


df_IS_EPI = df[df['IS-EPI'] != 0]
df_IS_HWI = df[df['IS-HWI'] != 0]
df_IS_IP = df[df['IS-IP'] != 0] 
df_IS_WSI = df[df['IS-WSI'] != 0]


In [ ]:
#join all dataframes together on 'TIME_PERIOD' and 'geo', drop row if missing values
df = pd.merge(df_IS_EPI, df_IS_HWI, on=['geo', 'TIME_PERIOD'], how='inner')
df.dropna(inplace=True)
#if value in 'IS-HWI_y' not = 0 set 'IS-HWI_x' = 'IS-HWI_y' 
df.loc[df['IS-HWI_y'] != 0, 'IS-HWI_x'] = df['IS-HWI_y']
df = df.iloc[:, :4]
#join all dataframes together on 'TIME_PERIOD' and 'geo', drop row if missing values
df = pd.merge(df, df_IS_IP, on=['geo', 'TIME_PERIOD'], how='inner')
df.dropna(inplace=True)
#if value in 'IS-HWI_y' not = 0 set 'IS-HWI_x' = 'IS-HWI_y' 
df.drop(['IS-EPI', 'IS-HWI', 'IS-WSI'], axis=1, inplace=True)
#join all dataframes together on 'TIME_PERIOD' and 'geo', drop row if missing values
df = pd.merge(df, df_IS_WSI, on=['geo', 'TIME_PERIOD'], how='inner')
df.dropna(inplace=True)
df.drop(['IS-EPI', 'IS-HWI', 'IS-IP_y'], axis=1, inplace=True)
#if colimn name has _x, remove _x
df.columns = df.columns.str.replace('_x', '')

df.head()



In [ ]:
df.info()

In [ ]:
#check for values = 0 in any column
df.isin([0]).any()

def indic(time, indic):
    #s = df_.loc['2010-Q1'] and where column = 'IS-EPI'
    s = df.loc[time, [indic,'geo']]
    #transpose s so geo is column names and TIME_PERIOD is index

    s = s.set_index('geo').T
    s= s.loc[indic]
    #drop rows where value is 0
    s = s[s != 0]
    return s
s = indic('2010-Q1', 'IS-EPI')


In [ ]:
#pip install bar_chart_race

In [ ]:
import plotly.express as px

df_ = px.data.gapminder()
df_.head()



In [ ]:
fig = px.bar(df_, x="geo", y="IS-HWI", color="IS-HWI",
  animation_frame="TIME_PERIOD", animation_group="geo")
fig.show()

In [ ]:
def cht(df, var, title):

    fig = px.bar(df, x="geo", y=var, color=var, range_y=[0, 400],
      animation_frame="TIME_PERIOD", animation_group="geo")
    fig.update_layout(title_text=title)
    fig.show(   )

vars = ['IS-EPI', 'IS-HWI', 'IS-IP', 'IS-WSI']

for var in vars:
    cht(df, var, var)


In [ ]:
def cht(df, var, title):

    fig = px.bar(df, x="geo", y=[var,'IS-EPI'], color=[var,'IS-EPI'], range_y=[0, 400],
      animation_frame="TIME_PERIOD", animation_group="geo")
    fig.update_layout(title_text=title)
    fig.show(   )
cht(df, 'IS-HWI', 'IS-HWI')

In [412]:
import plotly.express as px
import pandas as pd

# Sample data
data = pd.DataFrame({
    'Category': ['A', 'A', 'B', 'B', 'C', 'C'],
    'Value1': [10, 15, 12, 8, 9, 7],
    'Value2': [5, 7, 8, 6, 4, 5]
})
ie = df[df['geo'] == 'IE']['IS-HWI']
print(ie)
# Reshape the data to long format
data_long = pd.melt(df, id_vars='geo', value_vars=['IS-EPI', 'IS-HWI'], var_name='Value')

# Create grouped bar chart
fig = px.bar(data_long, x='geo', y='value', color='Value', barmode='group')

# Show the chart
fig.show()

729     92.2
730     83.1
731     74.0
732     80.8
733     78.7
734     79.9
735     84.7
736     79.8
737     75.5
738     69.2
739     72.4
740     74.7
741     66.4
742     75.5
743     78.4
744     75.3
745     76.8
746     93.9
747     92.1
748     94.5
749     96.4
750     98.1
751    102.2
752    103.4
753    104.1
754    117.9
755    120.5
756    119.5
757    125.3
758    121.2
759    126.4
760    125.5
761    129.6
762    143.7
763    149.7
764    144.7
765    135.6
766    146.0
767    150.8
768    150.9
769    155.9
770    102.9
771    143.5
772    146.8
773    124.0
774    126.7
775    153.7
776    176.0
777    167.9
778    172.1
779    183.5
780    174.6
Name: IS-HWI, dtype: float64
